## Dependency Linearization Playground

### Dependencies to latex

In [ ]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree

# ptb-dev path
deps_treebank = "/home/droca1/Treebanks/20ag/Penn-Treebank/ptb-dev.conllu"
trees = D_Tree.read_conllu_file(deps_treebank, filter_projective=True)

sample_tree=trees[554]
print(sample_tree)
print(D_Tree.to_latex(sample_tree))

<div style="text-align:center"><img src="./pics/notebooks/d_tree_1.png" /></div>

### Encode with 4-bits encoding

In [ ]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree

# ptb-dev path
deps_treebank = "/home/droca1/Treebanks/20ag/UD_English-EWT/en_ewt-ud-train.conllu"
trees = D_Tree.read_conllu_file(deps_treebank, filter_projective=False)
enc_7b = D_Brk7BitsEncoding(separator="_")
for i, sample_tree in enumerate(trees):
    lin_tree = enc_7b.encode(sample_tree)
    dec_tree = enc_7b.decode(lin_tree)
    las = dec_tree.las_score(sample_tree)
    
    if las != 1:
        print("Error at tree",i,"length",len(sample_tree))
        print(D_Tree.to_latex(sample_tree))
        print(lin_tree)
        print("LAS =",dec_tree.las_score(sample_tree))

In [ ]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree

deps_treebank = "/home/droca1/Treebanks/20ag/UD_English-EWT/en_ewt-ud-train.conllu"
trees = D_Tree.read_conllu_file(deps_treebank, filter_projective=False)
sample_tree = trees[6114]
enc_7b = D_Brk7BitsEncoding(separator="[_]")
lin_tree = enc_7b.encode(sample_tree)
print(lin_tree)
dec_tree = enc_7b.decode(lin_tree)
print("LAS =",dec_tree.las_score(sample_tree))

### Planar extraction for all UD Trees

In [26]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree
import os
import pandas as pd

# read all folders
ud_path="/home/droca1/Treebanks/20ag/"
ud_folders = [os.path.join(ud_path, f) for f in os.listdir(ud_path) if os.path.isdir(os.path.join(ud_path, f))]
results_df = pd.DataFrame(columns=["Corpus","n_trees","planarity","r_deps","l_deps","avg_dependants"])

for ud_folder in ud_folders:
    treebank_name = (ud_folder.split("/")[-1]).replace("_","-")
    
    # get all conllu files in ud_folder
    conllu_files = [os.path.join(ud_folder, f) for f in os.listdir(ud_folder) if f.endswith(".conllu")]
    total_trees = []
    for conllu_file in conllu_files:
        deps_treebank = os.path.join(ud_folder, conllu_file)
        trees = D_Tree.read_conllu_file(deps_treebank, filter_projective=False)
        total_trees += trees
    
    plan_perc = D_Tree.get_planarity_percentage(total_trees)
    r_deps, l_deps = D_Tree.get_dependency_direction_percentage(total_trees)
    avg_dependants = D_Tree.get_avg_dependants(total_trees)
    results_df = pd.concat([results_df, pd.DataFrame([[treebank_name, len(total_trees), plan_perc[0], r_deps, l_deps, avg_dependants]], 
                                                     columns=["Corpus","n_trees","planarity","r_deps","l_deps","avg_dependants"])], ignore_index=True)

results_df.sort_values(by=["planarity"], ascending=False)
print(results_df.sort_values(by=["planarity"], ascending=False).to_latex(index=False))

\begin{tabular}{llrrrr}
\toprule
                  Corpus & n\_trees &  planarity &   r\_deps &   l\_deps &  avg\_dependants \\
\midrule
           PENN-TREEBANK &   43948 &   0.998953 & 0.487397 & 0.512603 &        2.295559 \\
           UD-Hebrew-HTB &    6143 &   0.987140 & 0.463491 & 0.536509 &        2.301833 \\
            UD-Tamil-TTB &     600 &   0.983333 & 0.685628 & 0.314372 &        2.262076 \\
          UD-English-EWT &   16622 &   0.972747 & 0.572834 & 0.427166 &        2.530780 \\
            UD-Wolof-WTB &    2107 &   0.970100 & 0.482173 & 0.517827 &        2.519338 \\
          UD-Chinese-GSD &    4997 &   0.969382 & 0.616777 & 0.383223 &        2.426972 \\
          UD-Russian-GSD &    5030 &   0.938767 & 0.490234 & 0.509766 &        2.263321 \\
          UD-Finnish-TDT &   15136 &   0.938689 & 0.528854 & 0.471146 &        2.365725 \\
           UD-Uyghur-UDT &    3456 &   0.918403 & 0.667189 & 0.332811 &        2.216945 \\
           UD-German-GSD &   15590 &   0.906

/tmp/ipykernel_106939/3976968631.py:29: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df.sort_values(by=["planarity"], ascending=False).to_latex(index=False))


In [25]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree
import os
import copy
import pandas as pd

# read all folders
ud_path="/home/droca1/Treebanks/20ag/"
ud_folders = [os.path.join(ud_path, f) for f in os.listdir(ud_path) if os.path.isdir(os.path.join(ud_path, f))]
results_df = pd.DataFrame(columns=["Corpus","BRK","BRK2P","BRK4B","BRK7B"])

ebrk   = D_BrkBasedEncoding(separator="[_]", displacement   = True)
ebrk2p = D_Brk2PBasedEncoding(separator="[_]", displacement = True)
ebrk4b = D_Brk4BitsEncoding(separator="[_]")
ebrk7b = D_Brk7BitsEncoding(separator="[_]")

for ud_folder in ud_folders:
    treebank_name = (ud_folder.split("/")[-1]).replace("_","-")

    conllu_files = [os.path.join(ud_folder, f) for f in os.listdir(ud_folder) if f.endswith(".conllu")]
    total_trees = []
    
    total_labels = {"brk":[], "brk2p":[], "brk4b":[], "brk7b":[]}
    
    for conllu_file in conllu_files:
        deps_treebank = os.path.join(ud_folder, conllu_file)
        trees = D_Tree.read_conllu_file(deps_treebank, 
                                        filter_projective=False)
        for t in trees:
            total_labels["brk"]   += [str(lbl) for lbl in ebrk.encode(copy.deepcopy(t)).labels]
            total_labels["brk2p"] += [str(lbl) for lbl in ebrk2p.encode((copy.deepcopy(t))).labels]
            total_labels["brk4b"] += [str(lbl) for lbl in ebrk4b.encode((copy.deepcopy(t))).labels]
            total_labels["brk7b"] += [str(lbl) for lbl in ebrk7b.encode((copy.deepcopy(t))).labels]

    n_unique_brk   = len(set(total_labels["brk"]))
    n_unique_brk2p = len(set(total_labels["brk2p"]))
    n_unique_brk4b = len(set(total_labels["brk4b"]))
    n_unique_brk7b = len(set(total_labels["brk7b"]))

    results_df = pd.concat([results_df, pd.DataFrame([[treebank_name, n_unique_brk, n_unique_brk2p, n_unique_brk4b, n_unique_brk7b]],
                                                        columns=["Corpus","BRK","BRK2P","BRK4B","BRK7B"])], ignore_index=True)

print(results_df.to_latex(index=False))

\begin{tabular}{lllll}
\toprule
                  Corpus &  BRK & BRK2P & BRK4B & BRK7B \\
\midrule
          UD-Russian-GSD &  802 &   961 &   400 &   614 \\
          UD-Finnish-TDT & 1054 &  1223 &   435 &   685 \\
           UD-German-GSD & 1120 &  1476 &   428 &   771 \\
           PENN-TREEBANK & 1216 &  1233 &   396 &   408 \\
UD-Ancient-Greek-Perseus & 1469 &  2401 &   304 &  1167 \\
          UD-Chinese-GSD &  804 &   912 &   321 &   406 \\
           UD-Hebrew-HTB &  754 &   798 &   317 &   357 \\
            UD-Tamil-TTB &  262 &   274 &   153 &   164 \\
           UD-Uyghur-UDT &  553 &   683 &   353 &   475 \\
            UD-Wolof-WTB &  585 &   643 &   318 &   382 \\
          UD-English-EWT & 1089 &  1281 &   487 &   709 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_106939/3076216841.py:43: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df.to_latex(index=False))


### Hexatag

In [ ]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree
from codelin.models.const_tree import C_Tree
from nltk.tree import Tree

def pt(t, d=True):
    if d:
        if type(t) is list:
            for i in t:
                print(i)
            for i in t:
                Tree.fromstring(str(i)).pretty_print()
        else:
            Tree.fromstring(str(t)).pretty_print()

path = "/home/droca1/Treebanks/20ag/UD_English-EWT/en_ewt-ud-test.conllu"
encoder = D_Brk4BitsEncoding(separator = "[_]")
trees = D_Tree.read_conllu_file(path, filter_projective=True)
sample = trees[0]

print(sample)
bht = D_Tree.to_bht(sample)
print(bht)
dec_tree = D_Tree.from_bht(bht)
print(dec_tree)


### Clean multi-expression lines from CONLLU files

In [ ]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree
import os
import re

ag20_path="/home/droca1/Treebanks/20ag/"
ag20_folders = [os.path.join(ag20_path, f) for f in os.listdir(ag20_path) if os.path.isdir(os.path.join(ag20_path, f))]
mtl = [True, False]
encoder = D_Brk4BitsEncoding(separator="[_]")


for ag20_folder in ag20_folders:
    print("[INFO] Processing",ag20_folder)
    # get all conllu files
    treebank_name = (ag20_folder.split("/")[-1])
    conllu_files = [os.path.join(ag20_folder, f) for f in os.listdir(ag20_folder) if (f.endswith(".conllu"))]
    
    train_file = None
    dev_file = None

    # encode
    for conllu_file in conllu_files:
        print("[INFO] Cleaning",conllu_file)
        deps_treebank = os.path.join(ag20_folder, conllu_file)
        output_file = os.path.join(ag20_folder, conllu_file)
        
        with open(deps_treebank, "r") as f:
            lines = f.readlines()
        
        with open(deps_treebank, "w") as f:
            for line in lines:
                if re.match(r"^\d+-.*", line):
                    continue
                f.write(line)

### Encode and generate machamp config for training

In [ ]:
from codelin.encs.enc_deps import *
from codelin.models.deps_tree import D_Tree
import os

import json

config_singletask = {
    "dependency":{
        "train_data_path":"XXX",
        "dev_data_path":"XXX",
        "word_idx":0,
        "tasks":{
            "label":{
                "task_type":"seq",
                "column_idx":2
            }
        }
    }
}

config_multitask = {
    "dependency":{
        "train_data_path":"XXX",
        "dev_data_path":"XXX",
        "word_idx":0,
        "tasks":{
            "brk":{
                "task_type":"seq",
                "column_idx":2
            },
            "reltype":{
                "task_type":"seq",
                "column_idx":3
            }
        }
    }
}

ag20_path="/home/droca1/Treebanks/20ag/"
ag20_folders = [os.path.join(ag20_path, f) for f in os.listdir(ag20_path) if os.path.isdir(os.path.join(ag20_path, f))]
mtl = [True, False]

#encoder = D_Brk7BitsEncoding(separator="[_]")
encoder = D_Brk2PBasedEncoding(separator="[_]")

filter_projective = False
for ag20_folder in ag20_folders:
    print("[INFO] Processing",ag20_folder)
    # get all conllu files
    treebank_name = (ag20_folder.split("/")[-1])
    conllu_files = [os.path.join(ag20_folder, f) for f in os.listdir(ag20_folder) if (f.endswith(".conllu"))]

    
    train_file = None
    dev_file = None

    # encode
    for conllu_file in conllu_files:
        deps_treebank = os.path.join(ag20_folder, conllu_file)
        output_file = os.path.join(ag20_folder, conllu_file)
        target_extension = ".labels"
        output_file = output_file.replace(".conllu", target_extension)  
        
        if "train" in output_file:
            train_file = output_file
        elif "dev" in output_file:
            dev_file = output_file
        
        trees = D_Tree.read_conllu_file(deps_treebank, 
                                        filter_projective=filter_projective)

        with open(output_file, "w") as f:
            for tree in trees:
                lin_tree = encoder.encode(tree)
                f.write(lin_tree.to_string(f_idx_dict=None, 
                                           add_bos_eos=True, 
                                           separate_columns=True) +"\n")
        
        # save a clean test
        if 'test' in conllu_file and filter_projective:
            output_file = output_file.replace(".labels", "-clean.conllu")
            with open(output_file, "w") as f:
                for tree in trees:
                    tree.remove_dummy()
                    f.write("# text = "+tree.get_sentence()+"\n")
                    f.write(str(tree))
    
    current_config = config_multitask.copy()
    current_config["dependency"]["train_data_path"] = train_file.replace('droca1', 'diego.roca')
    current_config["dependency"]["dev_data_path"] = dev_file.replace('droca1', 'diego.roca')

    config_name = "config.json"
    with open(os.path.join(ag20_folder, config_name), "w") as f:
        json.dump(current_config, f, indent=4)